In [1]:
!nvidia-smi

Mon Oct 21 01:43:26 2024       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 528.79       Driver Version: 528.79       CUDA Version: 12.0     |
|-------------------------------+----------------------+----------------------+
| GPU  Name            TCC/WDDM | Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  NVIDIA GeForce ... WDDM  | 00000000:01:00.0 Off |                  N/A |
| N/A   46C    P8    N/A /  N/A |      0MiB /  4096MiB |      1%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

In [2]:
from transformers import pipeline, set_seed

import matplotlib.pyplot as plt

import pandas as pd
from datasets import load_dataset, load_metric

from transformers import AutoModelForSeq2SeqLM, AutoTokenizer

import nltk
from nltk.tokenize import sent_tokenize

from tqdm import tqdm
import torch

nltk.download("punkt")

c:\Users\Apoorv.DESKTOP-3VQL5IT\miniconda3\envs\textS\lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
[nltk_data] Downloading package punkt to C:\Users\Apoorv.DESKTOP-3VQL5
[nltk_data]     IT\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!


True

In [7]:
from datasets import load_dataset

# Load the CNN/DailyMail dataset (version 3.0.0)
dataset = load_dataset("cnn_dailymail", "3.0.0")

# Print the column names (features) of the training split
print(f"Features in cnn_dailymail: {dataset['train'].column_names}")

Features in cnn_dailymail: ['article', 'highlights', 'id']


In [8]:
sample = dataset["train"][1]
print(f"""
Article (excerpt of 500 characters, total length: {len(sample["article"])}):
""")
print(sample["article"][:500])
print(f'\nSummary (length: {len(sample["highlights"])}):')
print(sample["highlights"])


Article (excerpt of 500 characters, total length: 4051):

Editor's note: In our Behind the Scenes series, CNN correspondents share their experiences in covering news and analyze the stories behind the events. Here, Soledad O'Brien takes users inside a jail where many of the inmates are mentally ill. An inmate housed on the "forgotten floor," where many mentally ill inmates are housed in Miami before trial. MIAMI, Florida (CNN) -- The ninth floor of the Miami-Dade pretrial detention facility is dubbed the "forgotten floor." Here, inmates with the most s

Summary (length: 281):
Mentally ill inmates in Miami are housed on the "forgotten floor"
Judge Steven Leifman says most are there as a result of "avoidable felonies"
While CNN tours facility, patient shouts: "I am the son of the president"
Leifman says the system is unjust and he's fighting for change .


In [9]:
sample_text = dataset["train"][1]["article"][:1000]

# We'll collect the generated summaries of each model in a dictionary
summaries = {}

In [10]:
###Summarization Baseline, tokenization based on sentence.
def baseline_summary_three_sent(text):
    return "\n".join(sent_tokenize(text)[:3])

In [11]:
summaries['baseline'] = baseline_summary_three_sent(sample_text)

summaries['baseline']

'Editor\'s note: In our Behind the Scenes series, CNN correspondents share their experiences in covering news and analyze the stories behind the events.\nHere, Soledad O\'Brien takes users inside a jail where many of the inmates are mentally ill. An inmate housed on the "forgotten floor," where many mentally ill inmates are housed in Miami before trial.\nMIAMI, Florida (CNN) -- The ninth floor of the Miami-Dade pretrial detention facility is dubbed the "forgotten floor."'

In [12]:
from transformers import pipeline, set_seed

set_seed(42)

pipe = pipeline('text-generation', model = 'gpt2-medium' )

gpt2_query = sample_text + "\nTL;DR:\n" #TL, DR addded as we want to use GPT 2 for text summarization here.

pipe_out = pipe(gpt2_query, max_length = 512, clean_up_tokenization_spaces = True)

c:\Users\Apoorv.DESKTOP-3VQL5IT\miniconda3\envs\textS\lib\site-packages\huggingface_hub\file_download.py:157: UserWarning: `huggingface_hub` cache-system uses symlinks by default to efficiently store duplicated files but your machine does not support them in C:\Users\Apoorv.DESKTOP-3VQL5IT\.cache\huggingface\hub\models--gpt2-medium. Caching files will still work but in a degraded version that might require more space on your disk. This warning can be disabled by setting the `HF_HUB_DISABLE_SYMLINKS_WARNING` environment variable. For more details, see https://huggingface.co/docs/huggingface_hub/how-to-cache#limitations.
To support symlinks on Windows, you either need to activate Developer Mode or to run Python as an administrator. In order to see activate developer mode, see this article: https://docs.microsoft.com/en-us/windows/apps/get-started/enable-your-device-for-development
  warnings.warn(message)
c:\Users\Apoorv.DESKTOP-3VQL5IT\miniconda3\envs\textS\lib\site-packages\transformer

In [13]:
pipe_out

[{'generated_text': 'Editor\'s note: In our Behind the Scenes series, CNN correspondents share their experiences in covering news and analyze the stories behind the events. Here, Soledad O\'Brien takes users inside a jail where many of the inmates are mentally ill. An inmate housed on the "forgotten floor," where many mentally ill inmates are housed in Miami before trial. MIAMI, Florida (CNN) -- The ninth floor of the Miami-Dade pretrial detention facility is dubbed the "forgotten floor." Here, inmates with the most severe mental illnesses are incarcerated until they\'re ready to appear in court. Most often, they face drug charges or charges of assaulting an officer --charges that Judge Steven Leifman says are usually "avoidable felonies." He says the arrests often result from confrontations with police. Mentally ill people often won\'t do what they\'re told when police arrive on the scene -- confrontation seems to exacerbate their illness and they become more paranoid, delusional, and

In [15]:
len(gpt2_query)

1008

In [14]:
#text summarization using GPT2, we remove the input here so that we received only summarized output.
pipe_out[0]["generated_text"][len(gpt2_query) :]

'To get to the jail that holds the mentally ill, visit our Behind the Scenes blog -- Click here The story doesn\'t end there:\xa0 In 2014, a judge ordered the jail to provide treatment for 40 mental-health detainees in the mental-health unit, as part of a $22,000 settlement.\nInmates in the mental-health unit at Miami-Dade County\'s jail are often locked in a cell that\'s usually just like any other in the facility and can become chaotic. Mental health unit employees are often required to leave the jail and drive to the facility, instead of coming in to work with the inmates themselves. Most mental-health detainees spend hours a day sleeping in front of the wall by the pool, waiting for treatment to show up.\nThere are two more stories from the Inside the Tincup Jail series:\n\xa0\xa0\xa0 The "no contact" policy:\xa0 At an overcrowded state mental health facility in Miami-Dade County, the rules allow the police to try to convince the jail staff to allow a person they believe is mentall

In [16]:
summaries['gpt2'] = "\n".join(sent_tokenize(pipe_out[0]["generated_text"][len(gpt2_query) :]))


In [ ]:
#Using T5
pipe = pipeline('summarization', model = 't5-small' )

pipe_out = pipe(sample_text)